In [1]:
ls /kaggle/input/skin-condition-knowledge/JPEGImages-20250420T222156Z-001/JPEGImages

levle0_0.jpg    levle0_379.jpg  levle1_17.jpg   levle1_443.jpg  levle2_132.jpg
levle0_100.jpg  levle0_37.jpg   levle1_180.jpg  levle1_444.jpg  levle2_134.jpg
levle0_101.jpg  levle0_380.jpg  levle1_181.jpg  levle1_445.jpg  levle2_135.jpg
levle0_102.jpg  levle0_381.jpg  levle1_182.jpg  levle1_446.jpg  levle2_136.jpg
levle0_103.jpg  levle0_382.jpg  levle1_183.jpg  levle1_447.jpg  levle2_137.jpg
levle0_104.jpg  levle0_383.jpg  levle1_184.jpg  levle1_448.jpg  levle2_138.jpg
levle0_105.jpg  levle0_384.jpg  levle1_185.jpg  levle1_449.jpg  levle2_139.jpg
levle0_106.jpg  levle0_385.jpg  levle1_186.jpg  levle1_44.jpg   levle2_13.jpg
levle0_107.jpg  levle0_386.jpg  levle1_187.jpg  levle1_450.jpg  levle2_140.jpg
levle0_108.jpg  levle0_387.jpg  levle1_188.jpg  levle1_451.jpg  levle2_141.jpg
levle0_109.jpg  levle0_388.jpg  levle1_189.jpg  levle1_452.jpg  levle2_142.jpg
levle0_10.jpg   levle0_389.jpg  levle1_18.jpg   levle1_453.jpg  levle2_143.jpg
levle0_110.jpg  levle0_38.jpg   levle1_190.jpg  levle

In [2]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np


DEVICE = torch.device('cpu')
OUTPUT_SIZE = 2048

model = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.IMAGENET1K_V2)

extraction_layer = model._modules.get('avgpool')
model.to(DEVICE)
model.eval()

scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_vec(arg, model, extraction_layer):
    image = normalize(to_tensor(scaler(arg))).unsqueeze(0).to(DEVICE)
    result = torch.zeros(1, OUTPUT_SIZE, 1, 1)
    def copy_data(m, i, o):
        result.copy_(o.data)
    hooked = extraction_layer.register_forward_hook(copy_data)
    with torch.no_grad():
        model(image)
    hooked.remove()
    return result

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-1a0047aa.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-1a0047aa.pth
100%|██████████| 95.8M/95.8M [00:01<00:00, 72.0MB/s]


In [3]:
import arrow
import base64
import pandas as pd
from glob import iglob
from io import BytesIO
from os.path import basename
from PIL import Image

THUMBNAIL_SIZE = (64, 64)
TRAIN = '/kaggle/input/skin-condition-knowledge/JPEGImages-20250420T222156Z-001'


def embed(model, filename: str):
    with Image.open(fp=filename, mode='r') as image:
        return get_vec(arg=image.convert('RGB'), model=model, extraction_layer=extraction_layer).numpy().reshape(OUTPUT_SIZE,)


# https://stackoverflow.com/a/952952
def flatten(arg):
    return [x for xs in arg for x in xs]

def png(filename: str) -> str:
    with Image.open(fp=filename, mode='r') as image:
        buffer = BytesIO()
        # our images are pretty big; let's shrink the hover images to thumbnail size
        image.resize(size=THUMBNAIL_SIZE).convert('RGB').save(buffer, format='png')
        return 'data:image/png;base64,' + base64.b64encode(buffer.getvalue()).decode()

def get_picture_from_glob(arg: str, tag: str,) -> list:
    time_get = arrow.now()
    result = [pd.Series(data=[tag, basename(input_file), embed(model=model, filename=input_file), png(filename=input_file)],
                        index=['tag', 'name', 'value', 'png'])
        for index, input_file in enumerate(list(iglob(pathname=arg))) if input_file.endswith('.jpg') and index < 1000]
    print('encoded {} rows of {}  in {}'.format(len(result), tag, arrow.now() - time_get))
    return result

time_start = arrow.now()
train_dict = {basename(folder) : folder + '/*.*' for folder in iglob(TRAIN + '/*')}
train_df = pd.DataFrame(data=flatten(arg=[get_picture_from_glob(arg=value, tag=key) for key, value in train_dict.items()]))
print('done in {}'.format(arrow.now() - time_start))

encoded 1000 rows of JPEGImages  in 0:06:30.925531
done in 0:06:31.002432
